In [ ]:
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import Node2Vec
torch_geometric.__version__
import gc
import polars as pl

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
train_df = pl.read_parquet('../data/train.parquet',
                           columns=['session','aid'],
                           low_memory= True,
                          )
test_df = pl.read_parquet('../data/test.parquet',columns=['session','aid'])

In [ ]:
# 이전 세션(session)의 aid 값을 포함하는 그래프 데이터 저장

def lagged_df(df):
    # 이전 세션(session)의 aid 값을 새로운 열(column)로 추가
    df = df.with_column(pl.col("aid").shift(periods=1).over("session")
                              #.cast(pl.Int32)
                              #.fill_null(pl.col("aid"))
                              .alias("prev_aid"))
    return df
train_df = lagged_df(train_df)
test_df = lagged_df(test_df)

df = pl.concat([
    train_df,
    test_df
], how="vertical")
edges_torch_T = torch.tensor(np.transpose(df[['prev_aid','aid']].to_numpy()),dtype=torch.long)
torch.save(edges_torch_T,"all_edges_train_and_test.pt")

In [ ]:
edges_tensor = torch.load("../output/all_edges_train_and_test.pt")

In [ ]:
# 그래프 데이터 생성
data = Data(edge_index=edges_tensor)
# edges_tensor라는 텐서(tensor)를 이용하여 그래프 데이터를 생성
print(data)

Data 클래스는 그래프 데이터를 저장하는 데 사용되는 클래스.
edge_index 속성은 각 엣지(edge)의 시작 노드와 끝 노드를 표시하는 텐서(tensor).
edge_index 텐서를 이용하여 그래프의 연결 상태를 나타내는 그래프 데이터를 생성할 수 있습니다.

In [ ]:
del edges_tensor
gc.collect()

In [ ]:
# Embedding graph using Node2Vec

model = Node2Vec(data.edge_index, embedding_dim=32, 
                 walk_length=10,                        # lenght of rw
                 context_size=5, walks_per_node=10,
                 num_negative_samples=2, 
                 p=0.2, q=0.5,                             # bias parameters
                 sparse=True).to(device)

data.edge_index: 그래프에서의 엣지 정보를 나타내는 텐서.    
embedding_dim=32: 노드 임베딩 벡터의 차원 수를 정의.    
walk_length=10: Random Walk의 길이를 정의.    
context_size=5: Skip-Gram 모델에서 사용되는 문맥 크기를 정의.    
walks_per_node=10: 노드마다 실행되는 Random Walk의 횟수를 정의.    
num_negative_samples=2: Skip-Gram 모델에서 사용되는 음수 샘플링의 수를 정의.    
p=0.2, q=0.5: Node2Vec에서 사용되는 Random Walk의 바이어스 파라미터. p는 이전 노드로 돌아가기 쉬운 경로를 선호하는 경향이 있으며, q는 더 멀리 이동하는 경로를 선호하는 경향이 있다.    
sparse=True: 엣지 인덱스가 희소 텐서일 경우 성능을 향상시키기 위해 True로 설정.    
.to(device): 모델을 CPU 또는 GPU와 같은 디바이스로 이동.    

In [ ]:
loader = model.loader(batch_size=128, shuffle=True,
                      num_workers=2)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

In [ ]:
del data
gc.collect()

In [ ]:
 def train():
        model.train()
        total_loss = 0
        for pos_rw, neg_rw in tqdm(loader):
            optimizer.zero_grad()
            loss = model.loss(pos_rw.to(device), neg_rw.to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(loader)

for epoch in range(0, 12):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

In [ ]:
%%time
from annoy import AnnoyIndex

index = AnnoyIndex(32, 'angular')

for idx,idx_embedding in enumerate(model.state_dict()['embedding.weight'].cpu()):
    index.add_item(idx, idx_embedding)
    
index.build(10)

In [ ]:
embeddings_node2vec = model.cpu().state_dict()['embedding.weight'].numpy()

np.save("node2vec_embeddings",embeddings_node2vec)

In [ ]:
del model, loader, optimizer, embeddings_node2vec
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Validation / Inference

In [ ]:
def evaluate(path,mode="validation",n_neighbors=12):


    test = pl.read_parquet(path)

    session_types = ['clicks', 'carts', 'orders']
    test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
    test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)

    del test
    gc.collect()
    labels = []

    type_weight_multipliers = {0: 1, 1: 6, 2: 3}

    for AIDs, types in zip(test_session_AIDs, test_session_types):
        if len(AIDs) >= 20:
                # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
            weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
            aids_temp=defaultdict(lambda: 0)
            for aid,w,t in zip(AIDs,weights,types): 
                aids_temp[aid]+= w * type_weight_multipliers[t]

            sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
            labels.append(sorted_aids[:20])
        else:
            # here we don't have 20 aids to output -- we will use word2vec embeddings to generate candidates!
            AIDs = list(dict.fromkeys(AIDs[::-1]))

            # let's grab the most recent aid
            most_recent_aid = AIDs[0]

            # and look for some neighbors!
            nns = [i for i in index.get_nns_by_item(most_recent_aid, n_neighbors+1)[1:]]


            labels.append((AIDs+nns)[:n_neighbors])

    labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

    predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

    prediction_dfs = []

    for st in session_types:
        modified_predictions = predictions.copy()
        modified_predictions.session_type = modified_predictions.session_type.astype('str') + f'_{st}'
        prediction_dfs.append(modified_predictions)

    sub = pd.concat(prediction_dfs).reset_index(drop=True)
    
    del prediction_dfs, predictions,labels_as_strings, labels, test_session_types,test_session_AIDs
    gc.collect()
    if mode=="test":
        sub.to_csv("submission.csv",index=False)
        return sub
    else:

        sub['labels_2'] = sub['labels'].apply(lambda x : [int(s) for s in x.split(' ')])
        submission = pd.DataFrame()
        submission['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
        submission['type'] = sub.session_type.apply(lambda x: x.split('_')[1])
        submission['labels'] = sub.labels_2.apply(lambda x : [item for item in x[:] ]) #.apply(lambda x: [int(i) for i in x.split(',')[:20]])
        test_labels = pd.read_parquet('/kaggle/input/otto-train-and-test-data-for-local-validation/test_labels.parquet')
        test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
        del sub,submission
        gc.collect()
        gc.collect()
        test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
        test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
        recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 
        score = (recall_per_type * pd.Series({'clicks': 0.1, 'carts': 0.30, 'orders': 0.60})).sum()

        return score

In [ ]:
path = "/kaggle/input/otto-train-and-test-data-for-local-validation/test.parquet"
validation_score = evaluate(path,mode="validation",n_neighbors=20)

In [ ]:
path = "/kaggle/input/otto-full-optimized-memory-footprint/test.parquet"
test_submission = evaluate(path,mode="test",n_neighbors=20)